# Как применять модель с использованием HF

Модель скачивается автоматически из моего репозитория на HaggingFace
При желании можно скачать самостоятельно [отсюда](https://huggingface.co/data-silence/lstm-news-classifier/resolve/main/model.pth?download=true)

In [ ]:
import torch.nn as nn
from transformers import BertModel
import torch
from transformers import AutoTokenizer
from huggingface_hub import hf_hub_download

## 1. Задаём класс для классификатора

(без этого шага загруженная в новую среду модель не сможет инициализировать свои параметры и выдаст ошибку)

In [ ]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, hidden_dim, output_dim, n_layers, dropout):
        super(BiLSTMClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.lstm = nn.LSTM(self.bert.config.hidden_size, hidden_dim, num_layers=n_layers,
                            bidirectional=True, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask, labels=None):
            with torch.no_grad():
                embedded = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
            lstm_out, _ = self.lstm(embedded)
            pooled = torch.mean(lstm_out, dim=1)
            logits = self.fc(self.dropout(pooled))

            if labels is not None:
                loss_fn = nn.CrossEntropyLoss()
                loss = loss_fn(logits, labels)
                return {"loss": loss, "logits": logits}  # Возвращаем словарь
            return logits  # Возвращаем логиты, если метки не переданы

In [ ]:
# словарь для расшифровки категорий, т.к. модель предскажет только номер класса
categories = ['climate', 'conflicts', 'culture', 'economy', 'gloss', 'health',
              'politics', 'science', 'society', 'sports', 'travel']

# загружаем модель с HF
repo_id = "data-silence/lstm-news-classifier"
tokenizer = AutoTokenizer.from_pretrained(repo_id)
model_path = hf_hub_download(repo_id=repo_id, filename="model.pth")

model = torch.load(model_path)

# получаем предсказания модели
def get_predictions(news: str, model) -> str:
    with torch.no_grad():
        inputs = tokenizer(news, return_tensors="pt")
        del inputs['token_type_ids']
        output = model.forward(**inputs)
    id_best_label = torch.argmax(output[0, :], dim=-1).detach().cpu().numpy()
    prediction = categories[id_best_label]
    return prediction

C:\Users\user\AppData\Local\Temp\ipykernel_24764\1387115929.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


In [ ]:
# Использование классификатора
news = 'В Париже завершилась церемония завершения Олимпийский игр'
get_predictions(news, model=model)

'sports'